In [1]:
# Author: Dan Schelkoph
# https://github.com/dschelkoph

import abc
from dataclasses import dataclass
from typing import Annotated, TypeAlias, get_type_hints

from pydantic import AfterValidator, BaseModel, Field, field_validator

# ruff: noqa: D100, D101, D102, D103, D105, TRY003

# What is the `Annotated` Type?

## Notes

[Python Docs](https://docs.python.org/3/library/typing.html#typing.Annotated)

Allows the addition of metadata to a type annotation.

The type has arguments that look like this: `Annotated[<Acutual Type>, <Metadata>, ...]`. Each metadata argument can be any type.

Unique in that the type system doesn't see objects as the `Annotated` type, but whatever the first argument is. To use the metadata, your program has to purposely retrieve it.

## Code

In [2]:
annotated_int: Annotated[int, "this is metadata", "this is more metadata"] = 2
print(f"Type of `annotated_int`: {type(annotated_int)}")

Type of `annotated_int`: <class 'int'>


### Passing Into Function

Since the type system sees `annotated_int` as an `int`, there isn't a type error passing it into `add_two`.

In [3]:
def add_two(addend: int) -> int:
    return addend + 2


add_two(annotated_int)

4

### `Annotated` in Function

In addition, Non-Annotated types can be passed into objects that use `Annotated` type hints.

In [4]:
unannotated_int: int = 2
print(f"Type of `annotated_int`: {type(unannotated_int)}")

Type of `annotated_int`: <class 'int'>


In [5]:
def annotated_add_two(addend: Annotated[int, "this is metadata"]):
    return addend + 2


annotated_add_two(unannotated_int)

4

Since the type system only uses the first argument, it allows the usage of the `Annotated` anywhere in your code without creating breaking changes to your typing!

# Where's the Metadata?

## Notes

Can obtain through the `__metadata__` attribute of the `Annotated` type.

To obtain typing information with metadata from an object, can use `get_type_hints` with the ``

## Code

In [6]:
AnnotatedInt: TypeAlias = Annotated[int, "this is metadata"]

Use the `__metadata__` attribute:

In [13]:
print(f"Annotated Metadata: {AnnotatedInt.__metadata__}")
print(f"Normal `int` Metadata: {getattr(int, "__metadata__", "`__metadata__` Doesn't exist.")}")

Annotated Metadata: ('this is metadata',)
Normal `int` Metadata: `__metadata__` Doesn't exist.


Use `get_type_hints`:

In [8]:
@dataclass
class Integers:
    first_num: AnnotatedInt
    second_num: int


get_type_hints(Integers)

{'first_num': int, 'second_num': int}

There is no metadata though...

In [9]:
# Must use `include_extras=True` to get metadata (for backwards compatibility)
type_hints = get_type_hints(Integers, include_extras=True)
print(type_hints)

{'first_num': typing.Annotated[int, 'this is metadata'], 'second_num': <class 'int'>}


In [10]:
type_hints["first_num"].__metadata__

('this is metadata',)

# So What?

## Validator

In [11]:
class Validator(abc.ABC):
    """Concrete implementations can be placed as metadata in `Annotated` to validate a type."""

    @abc.abstractmethod
    def validate(self, param_name: str, value) -> bool:
        ...


class PositiveIntValidator(Validator):
    """Ensure an integer is positive."""

    def validate(self, param_name: str, value: int) -> bool:
        if value <= 0:
            raise ValueError(f"`{param_name}` must be positive. Current value: {value}.")
        return True


@dataclass
class Square:
    side: Annotated[int, PositiveIntValidator(), "its hip to be square"]

    def __post_init__(self):
        # type_hints is `dict[ParamName, type]`
        type_hints = get_type_hints(self, include_extras=True)
        for param_name, param_type in type_hints.items():
            metadata_tuple = getattr(param_type, "__metadata__", ())
            for metadata in metadata_tuple:
                # we only care about metadata that inherits from `Validator`
                if isinstance(metadata, Validator):
                    metadata.validate(param_name, getattr(self, param_name))

In [12]:
Square(-1)

ValueError: `side` must be positive. Current value: -1.

In [ ]:
Square(2)

Square(side=2)

# Pydantic Usage

## Notes


Let's take a look at how `pydantic` can use the `Annotated` type.

Here are the links to the pydantic docs (v2):
- [Field Definition](https://docs.pydantic.dev/latest/concepts/fields/#using-annotated)
- [Validation](https://docs.pydantic.dev/latest/concepts/validators/#annotated-validators)

Putting the `Field Definition` in the type hint makes assigning a default value like standard python.

By defining validation in a TypeAlias, we can use that type in many different models and get the same validation as opposed to a [Field Validator](https://docs.pydantic.dev/latest/concepts/validators/#field-validators) that must be defined in every class unless inheritance is used.

The tradeability between readability and reuse may not be worth it in every case!

## Pydantic

In [ ]:
def ensure_even(v: int) -> int:
    if v % 2 != 0:
        raise ValueError("Number must be even.")
    return v


# can nest annotated tyes
PositiveInt: TypeAlias = Annotated[int, Field(gt=0)]
PositiveEvenInt: TypeAlias = Annotated[PositiveInt, AfterValidator(ensure_even)]


class EvenRectangleAnnotations(BaseModel):
    """Rectangle where length and height are both even integers."""

    length: PositiveEvenInt
    # height has the same metadata as `PositiveEvenInt`
    height: Annotated[int, Field(gt=0), AfterValidator(ensure_even)]

In [ ]:
EvenRectangleAnnotations(length=-2, height=5)

ValidationError: 2 validation errors for EvenRectangleAnnotations
length
  Input should be greater than 0 [type=greater_than, input_value=-2, input_type=int]
    For further information visit https://errors.pydantic.dev/2.5/v/greater_than
height
  Value error, Number must be even. [type=value_error, input_value=5, input_type=int]
    For further information visit https://errors.pydantic.dev/2.5/v/value_error

In [ ]:
EvenRectangleAnnotations(length=2, height=4)

EvenRectangleAnnotations(length=2, height=4)

Using `Annotated` and `TypeAliases` can reuse more code when the same types are needed multiple times.

In [ ]:
class PyramidAnnotations(BaseModel):
    length: PositiveInt
    width: PositiveInt
    height: PositiveInt